### tf.summary
> Operations for writing summary data, for use in analysis and visualization.

In this jupyter notebook we will learning these fucntion of tf.summary:
1. tf.summary.create_file_writer()
3. tf.summary.histogram()
4. tf.summary.graph()
5. tf.summary.trace_export()
6. tf.summary.trace_on()
7. tf.summary.trace_off()


In [1]:
import tensorflow as tf 
from datetime import datetime
%load_ext tensorboard

2022-10-20 22:29:31.346341: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-20 22:29:31.523440: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-10-20 22:29:31.617713: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-20 22:29:32.262793: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: li

In [2]:
## Defining a basic tf module
class tfDense(tf.Module):
    def __init__(self, out_features,name=None):
        super().__init__(name)
        self.out_features=out_features
        
        self.is_build=False
    @tf.function    
    def __call__(self,x):
        if not self.is_build:
            self.w=tf.Variable(tf.random.normal([x.shape[-1],self.out_features]),name='w')
            self.b=tf.Variable(tf.zeros(self.out_features),name='b')
            self.is_build=True
        y=tf.matmul(x,self.w)+self.b
        return tf.nn.relu(y)

In [3]:
## Defining the model
x=tf.constant([[1.3,2.5]])
model=tfDense(out_features=5)

2022-10-20 22:29:35.403519: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-20 22:29:35.437655: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-20 22:29:35.437977: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-20 22:29:35.439507: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compi

tf.summary.create_file_writer
> Creates a summary file writer for the given log directory

**Returns a summary writer objects**

In [4]:
# Set up logging.
stamp = datetime.now().strftime("%Y%m%d-%H%M%S")
logdir = "logs/func/%s" % stamp
writer = tf.summary.create_file_writer(logdir) ## Creates a summary file writer int his given directory

tf.summary.trace_on()
> start a trace to record compytation graph and profiling information

Args
1. graph: default True. If true,enables collection of  executed graphs,which also includes one from tf.function
2. profiler: default False.  Enabling profiler implicitly enables the graph collection. The profiler may incur a high memory overhead.
> 


In [5]:
tf.summary.trace_on(graph=True)
tf.profiler.experimental.start(logdir) ## start profiling the tensorflow performance
#A profile is a set of statistics that describes how often and for how long various parts of the program executed.

2022-10-20 22:29:42.368119: I tensorflow/core/profiler/lib/profiler_session.cc:101] Profiler session initializing.
2022-10-20 22:29:42.368148: I tensorflow/core/profiler/lib/profiler_session.cc:116] Profiler session started.
2022-10-20 22:29:42.368975: I tensorflow/core/profiler/backends/gpu/cupti_tracer.cc:1664] Profiler found 1 GPUs
2022-10-20 22:29:42.369175: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcupti.so.11.2'; dlerror: libcupti.so.11.2: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/ros/noetic/lib:/usr/local/cuda-11.6/lib


Methods of **Summary writer object** (return from tf.summary.create_file_writer)
> ***writer*** is the Summary writer object in above function

writer.**as_default(step=None)** is one of the methods of **writer** object.
> Returns a context manager that enables summary writing.
For convenience, if step is not None, this function also sets a default value for the step parameter used in summary-writing functions elsewhere in the API so that it need not be explicitly passed in every such invocation. The value can be a constant or a variable.




### tf.summary.trace_export

>stops and export the active trace as Summary and/or profile file.

**Args**
1. name : A name for the the summary to be written.
2. step : Explicit int64-castable monotonic step value for this summary. If omitted, this defaults to tf.summary.experimental.get_step(), which must not be None.
3. profiler_outdir : Output directory for profiler. It is required when profiler is enabled when trace was started. Otherwise, it is ignored.

In [6]:
z=print(model(x))
with writer.as_default():
    tf.summary.trace_export(name='Dense_trace',
    step=0,
    profiler_outdir=logdir)

tf.Tensor([[0.02910441 0.         0.         1.6036919  0.        ]], shape=(1, 5), dtype=float32)


## tf.summary.histogram

In [7]:
w = tf.summary.create_file_writer('test/logs')
with w.as_default():
    tf.summary.histogram("activations", tf.random.uniform([100, 50]), step=0)
    tf.summary.histogram("initial_weights", tf.random.normal([1000]), step=0)

In [10]:
w = tf.summary.create_file_writer('test/logs')
with w.as_default():
    for step in range(100):
        # Generate fake "activations".
        activations = [
            tf.random.normal([1000], mean=step, stddev=1),
            tf.random.normal([1000], mean=step, stddev=10),
            tf.random.normal([1000], mean=step, stddev=100),
        ]

        tf.summary.histogram("layer1/activate", activations[0], step=step)
        tf.summary.histogram("layer2/activate", activations[1], step=step)
        tf.summary.histogram("layer3/activate", activations[2], step=step)

**To see the tensorboard logs of histogram use the folling commond**
<br>
tensorboard --logdir='test/logs/'